In [1]:
import pandas as pd
import json
import numpy as np

from kaggle_environments.envs.football.helpers import Action

pd.set_option("display.float_format", lambda x: "%.5f" % x)
pd.options.display.max_rows = 999
pd.set_option('display.max_columns', 150)

np.set_printoptions(suppress=True)



# dictionary of sticky actions
sticky_actions = {
    "left": Action.Left,
    "top_left": Action.TopLeft,
    "top": Action.Top,
    "top_right": Action.TopRight,
    "right": Action.Right,
    "bottom_right": Action.BottomRight,
    "bottom": Action.Bottom,
    "bottom_left": Action.BottomLeft,
    "sprint": Action.Sprint,
    "dribble": Action.Dribble
}

action_set_dic = {
    0: 'idle',
    # movement actions (1-8)
    1: 'left', # run to the left, sticky action.
    2: 'top_left', #run to the top-left, sticky action.
    3: 'top', # run to the top, sticky action.
    4: 'top_right', # run to the top-right, sticky action.
    5: 'right', # run to the right, sticky action.
    6: 'bottom_right', # run to the bottom-right, sticky action.
    7: 'bottom', # run to the bottom, sticky action.
    8: 'bottom_left', # run to the bottom-left, sticky action
    
    # passing / shooting (9-12)
    9: 'long_pass', # perform a long pass to the player on your team. Player to pass the ball to is auto-determined based on the movement direction.
    10: 'high_pass', # perform a high pass, similar to long_pass.
    11: 'short_pass', # perform a short pass, similar to long_pass.
    12: 'shot', # perform a shot, always in the direction of the opponent's goal.
    
    13: 'sprint', # start sprinting, sticky action. Player moves faster, but has worse ball handling.
    14: 'release_direction', # reset current movement direction.
    15: 'release_sprint', # stop sprinting
    16: 'sliding', # perform a slide (effective when not having a ball)
    17: 'dribble', # start dribbling (effective when having a ball), sticky action. Player moves slower, but it is harder to take over the ball from him.
    18: 'release_dribble' # stop dribbling
}


# game_mode - current game mode, one of:
game_mode_dic = {
    0 : 'normal',
    1 : 'kickoff',
    2 : 'goalkick',
    3 : 'freekick',
    4 : 'corner',
    5 : 'throwin',
    6 : 'penalty'
}

# player roles
player_role_dic = {
    0: 'GK',
    1: 'CB',
    2: 'LB',
    3: 'RB',
    4: 'DM',
    5: 'CM',
    6: 'LM',
    7: 'RM',
    8: 'AM',
    9: 'CF'
}


def do_flatten(obj):
    if type(obj) == list:
        return np.array(obj).flatten()
    return obj.flatten()

def convert_observation(observation, fixed_positions=False):

    final_obs = []
    
    for obs in observation:

        o = []
        if fixed_positions:
            for i, name in enumerate(['left_team', 'left_team_direction',
                                    'right_team', 'right_team_direction']):
                o.extend(do_flatten(obs[name]))
            # If there were less than 11vs11 players we backfill missing values
            # with -1.
            if len(o) < (i + 1) * 22:
                o.extend([-1] * ((i + 1) * 22 - len(o)))
        else:
            o.extend(do_flatten(obs['left_team']))
            o.extend(do_flatten(obs['left_team_direction']))
            o.extend(do_flatten(obs['right_team']))
            o.extend(do_flatten(obs['right_team_direction']))

        # If there were less than 11vs11 players we backfill missing values with
        # -1.
        # 88 = 11 (players) * 2 (teams) * 2 (positions & directions) * 2 (x & y)
        if len(o) < 88:
            o.extend([-1] * (88 - len(o)))

        # ball position
        o.extend(obs['ball'])
        # ball direction
        o.extend(obs['ball_direction'])
        # one hot encoding of which team owns the ball
        if obs['ball_owned_team'] == -1:
            o.extend([1, 0, 0])
        if obs['ball_owned_team'] == 0:
            o.extend([0, 1, 0])
        if obs['ball_owned_team'] == 1:
            o.extend([0, 0, 1])

        active = [0] * 11
        if obs['active'] != -1:
            active[obs['active']] = 1
        o.extend(active)

        game_mode = [0] * 7
        game_mode[obs['game_mode']] = 1
        o.extend(game_mode)
        final_obs.append(o)

        return np.array(final_obs, dtype=np.float32).flatten()

    
def get_episode_simple115_v2_df(team_want, episode_id, episode_dir):
    # get json file for specific episode
    epsiode_full_dir = '{0}{1}.json'.format(episode_dir, episode_id)
    epsiode_full_dir

    with open(epsiode_full_dir) as json_file:
        obs = json.load(json_file)
        
    lteam = obs['info']['TeamNames'][0]
    rteam = obs['info']['TeamNames'][1]
    # want to get left or right based on agent
    if team_want ==  lteam:
        lr_index = 0
    else:
        lr_index = 1

    # want to get left or right based on agent
    if team_want ==  lteam:
        lr_index = 0
    else:
        lr_index = 1

    
    df = get_simple115_v2_df(obs)
    
    return df    


def get_simple115_v2_df(obs):
    steps = obs['steps']
    obs_list = []
    for step_num in range(len(steps)):
        observation = [steps[step_num][0]['observation']['players_raw'][0]]
        v2_115_obs = convert_observation(observation)
        obs_list.append(v2_115_obs)
       
    df_cols = get_simple115_v2_df_cols()
    df = pd.DataFrame(obs_list, columns=df_cols)
    return df



def get_simple115_v2_df_cols():
        # coords of left_team players (22)
    lcoords_col = []
    for i in range(11):
        lcoords_col.append('l_x' + str(i))
        lcoords_col.append('l_y' + str(i))
    # direction of left_team players (22)
    ldirs_col = []
    for i in range(11):
        ldirs_col.append('l_x_dir' + str(i))
        ldirs_col.append('l_y_dir' + str(i))
    # coords of right_team players (22)
    rcoords_col = []
    for i in range(11):
        rcoords_col.append('r_x' + str(i))
        rcoords_col.append('r_y' + str(i))
    # direction of left_team players (22)
    rdirs_col = []
    for i in range(11):
        rdirs_col.append('r_x_dir' + str(i))
        rdirs_col.append('r_y_dir' + str(i))
    # ball position (x,y,z)
    ball_pos = ['ball_x_pos', 'ball_y_pos', 'ball_z_pos']
    # ball direction (x,y,z)
    ball_dir = ['ball_x_dir', 'ball_y_dir', 'ball_z_dir']
    # one hot encoding of ball ownership (noone, left, right) (3)
    ball_own = ['ball_own_noone', 'ball_own_left', 'ball_own_right']
    # one hot encoding of which player is active (11)
    player_actives = ['p_active' + str(x) for x in range(11)]
    # one hot encoding of game mode (7)
    game_modes = ['gmode' + str(x) for x in range(7)]

    final_col_list = lcoords_col + ldirs_col + rcoords_col + rdirs_col + ball_pos + \
        ball_dir + ball_own + player_actives + game_modes
    
    return final_col_list


def get_episode_all_df(team_want, episode_id, episode_dir):
    
    # get json file for specific episode
    epsiode_full_dir = '{0}{1}.json'.format(episode_dir, episode_id)
    epsiode_full_dir

    with open(epsiode_full_dir) as json_file:
        obs = json.load(json_file)
      
    lteam = obs['info']['TeamNames'][0]
    rteam = obs['info']['TeamNames'][1]
    # want to get left or right based on agent
    if team_want ==  lteam:
        lr_index = 0
        agent_type = 'left'
    else:
        lr_index = 1
        agent_type = 'right'
    
    #create basic df
    steps = obs['steps']
    steps_lists = []

    for step_num in range(len(steps)):
        #print(i)
        step = steps[step_num]
        # get left or right player based on if matches team we want
        obs_step = step[lr_index]

        if step_num == 0:
            action = None
            action_str = None
        else:
            action = obs_step['action'][0]
            action_str = action_set_dic[action]
        

        status = obs_step['status']
        observation = obs_step['observation']
        players_raw = observation['players_raw'][0]
        
        active_player = players_raw['active']
        
        game_mode = players_raw['game_mode']
        game_mode_str = game_mode_dic[game_mode]

        # score
        left_score = players_raw['score'][0]
        right_score = players_raw['score'][1]

        # steps
        steps_left = players_raw['steps_left']
        
        # ball owned team  {-1, 0, 1}, -1 = ball not owned, 0 = left team, 1 = right team.
        ball_owned_team = players_raw['ball_owned_team']
        # we need to map ball owned based on if left or right agent
        if ball_owned_team == -1:
            off_def_flag = 'none'
        elif ball_owned_team == 0:
            off_def_flag = 'offense' if agent_type == 'left' else 'defense'
        elif ball_owned_team == 1:
            off_def_flag = 'offense' if agent_type == 'right' else 'defense'    

        # now create dataframe
        step_list = [step_num, action, action_str, game_mode_str, active_player, off_def_flag, left_score, right_score, agent_type, status]
        steps_lists.append(step_list)

        
    df_columns = ['step_num', 'action', 'action_str', 'game_mode_str', 'active_player', 'off_def_flag', 'left_score', 'right_score', 'agent_type', 'status']
    basic_df = pd.DataFrame(steps_lists, columns = df_columns)
    
    # get sidmple115_v2 embed df
    s115_df = get_simple115_v2_df(obs)
    s115_df['step_num'] = range(0, 3002)
    
    #merge
    final_df = pd.merge(basic_df, s115_df, how='outer', on=['step_num'])
    return final_df 

Using rogbas great episode scraper notebook https://www.kaggle.com/robga/google-football-episode-scraper we can use LightGBM to predict the top players actions.

I have an a couple agents nearly identical to this notebook that have scores over 940.



In [2]:
import pandas as pd
import numpy as np
import os
import requests
import json
import datetime
import time

In [3]:

import json
import os
from tqdm import tqdm, notebook

In [4]:
def convert_observation(observation, fixed_positions=False):

    def do_flatten(obj):
        if type(obj) == list:
            return np.array(obj).flatten()
        return obj.flatten()

    final_obs = []
    
    for obs in observation:
        

        o = []
        if fixed_positions:
            for i, name in enumerate(['left_team', 'left_team_direction',
                                    'right_team', 'right_team_direction']):
                o.extend(do_flatten(obs[name]))
            # If there were less than 11vs11 players we backfill missing values
            # with -1.
            if len(o) < (i + 1) * 22:
                o.extend([-1] * ((i + 1) * 22 - len(o)))
        else:
            o.extend(do_flatten(obs['left_team']))
            o.extend(do_flatten(obs['left_team_direction']))
            o.extend(do_flatten(obs['right_team']))
            o.extend(do_flatten(obs['right_team_direction']))

        # If there were less than 11vs11 players we backfill missing values with
        # -1.
        # 88 = 11 (players) * 2 (teams) * 2 (positions & directions) * 2 (x & y)
        if len(o) < 88:
            o.extend([-1] * (88 - len(o)))

        # ball position
        o.extend(obs['ball'])
        # ball direction
        o.extend(obs['ball_direction'])
        # one hot encoding of which team owns the ball
        if obs['ball_owned_team'] == -1:
            o.extend([1, 0, 0])
        if obs['ball_owned_team'] == 0:
            o.extend([0, 1, 0])
        if obs['ball_owned_team'] == 1:
            o.extend([0, 0, 1])

        active = [0] * 11
        if obs['active'] != -1:
            active[obs['active']] = 1
        o.extend(active)

        game_mode = [0] * 7
        game_mode[obs['game_mode']] = 1
        o.extend(game_mode)
        final_obs.append(o)

        return np.array(final_obs, dtype=np.float32).flatten()


In [5]:
# JSON_EPISODE_DIR = './../episode_scraping/episodes_dl/WeKick/sub_id_17747116/11-02-2020-15-47-08/'

# y=[]
# x=[]
# filenames = [p for p in os.listdir(JSON_EPISODE_DIR) if 'info' not in p and 'json' in p]

In [6]:
# %%time

# y=[]
# x=[]
# for f in filenames[1:3]:
#     print(f)
#     filename = JSON_EPISODE_DIR + f

#     try:
#         with open(filename) as json_file:
#             data = json.load(json_file)
#     except:
#         print('exception')
#         continue
        
#     counter=0


#     for team in [0,1]:
#         final_score = data['steps'][-2][team]['observation']['players_raw'][0]['score'][0]

#         goal=False

#         for i in range(2,len(data['steps'])-2,):

#             action=data['steps'][i][team]['action']
#             y.append(action[0])
            
#             obs=data['steps'][i][team]['observation']['players_raw'][0]
#             x.append(convert_observation([obs]))
                

## Read in episode Dataframes

We want to read in the dataframe to train against from the episodes



In [7]:
team_want = 'WeKick'
episode_dir = '../episode_scraping/episodes_dl/WeKick/sub_id_17747116/11-02-2020-15-47-08/'

In [8]:
episode_ids = []
json_episodes = os.listdir(episode_dir)
#print(len(json_episodes))
# we want to extract episodes from json file name
for episode_json in  json_episodes:
    #print(episode_json)
    if episode_json[-5:] == '.json' and episode_json != 'config.json':
        episode_ids.append(int(episode_json[:-5]))

In [9]:
print(len(episode_ids))

57


In [10]:
%%time
df_list = []
for episode_id in episode_ids:
    #print(episode_id)
    adf = get_episode_all_df(team_want, episode_id, episode_dir)
    adf['episode_id'] = episode_id
    df_list.append(adf)

CPU times: user 1min 15s, sys: 1.48 s, total: 1min 16s
Wall time: 1min 17s


In [11]:
fdf = pd.concat(df_list)
print(fdf.shape)
fdf.head(2)

(171114, 126)


,step_num,action,action_str,game_mode_str,active_player,off_def_flag,left_score,right_score,agent_type,status,l_x0,l_y0,l_x1,l_y1,l_x2,l_y2,l_x3,l_y3,l_x4,l_y4,l_x5,l_y5,l_x6,l_y6,l_x7,l_y7,l_x8,l_y8,l_x9,l_y9,l_x10,l_y10,l_x_dir0,l_y_dir0,l_x_dir1,l_y_dir1,l_x_dir2,l_y_dir2,l_x_dir3,l_y_dir3,l_x_dir4,l_y_dir4,l_x_dir5,l_y_dir5,l_x_dir6,l_y_dir6,l_x_dir7,l_y_dir7,l_x_dir8,l_y_dir8,l_x_dir9,l_y_dir9,l_x_dir10,l_y_dir10,r_x0,r_y0,r_x1,r_y1,r_x2,r_y2,r_x3,r_y3,r_x4,r_y4,r_x5,r_y5,r_x6,r_y6,r_x7,r_y7,r_x8,r_y8,r_x9,r_y9,r_x10,r_y10,r_x_dir0,r_y_dir0,r_x_dir1,r_y_dir1,r_x_dir2,r_y_dir2,r_x_dir3,r_y_dir3,r_x_dir4,r_y_dir4,r_x_dir5,r_y_dir5,r_x_dir6,r_y_dir6,r_x_dir7,r_y_dir7,r_x_dir8,r_y_dir8,r_x_dir9,r_y_dir9,r_x_dir10,r_y_dir10,ball_x_pos,ball_y_pos,ball_z_pos,ball_x_dir,ball_y_dir,ball_z_dir,ball_own_noone,ball_own_left,ball_own_right,p_active0,p_active1,p_active2,p_active3,p_active4,p_active5,p_active6,p_active7,p_active8,p_active9,p_active10,gmode0,gmode1,gmode2,gmode3,gmode4,gmode5,gmode6,episode_id
0,0,nan,None,normal,6,none,0,0,left,ACTIVE,-1.01103,-0.00000,-0.42665,-0.19894,-0.50551,-0.06459,-0.50551,0.06459,-0.42665,0.19894,-0.18624,-0.10740,-0.27053,-0.00000,-0.18624,0.10740,-0.01011,-0.21962,-0.05055,-0.00000,-0.01011,0.21962,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,1.01103,0.00000,0.42665,0.19894,0.50551,0.06459,0.50551,-0.06459,0.42665,-0.19894,0.18624,0.10740,0.27053,0.00000,0.18624,-0.10740,0.01011,0.21962,-0.00000,-0.02033,-0.00000,0.02033,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,0.00000,-0.00000,0.11062,0.00000,-0.00000,0.00616,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,4080761
1,1,4.00000,top_right,normal,9,none,0,0,left,ACTIVE,-1.01096,0.00000,-0.42665,-0.19894,-0.50386,-0.06441,-0.50551,0.06459,-0.42665,0.19894,-0.18525,-0.10683,-0.27053,-0.00000,-0.18624,0.10740,-0.00767,-0.21971,-0.04809,0.00000,-0.00993,0.21962,0.00066,0.00000,0.00000,-0.00000,0.00353,0.00035,0.00000,-0.00000,0.00000,-0.00000,0.00256,0.00148,0.00000,-0.00000,0.00000,-0.00000,0.00392,-0.00019,0.00399,0.00000,0.00128,0.00003,1.01103,0.00000,0.42665,0.19894,0.50532,0.06457,0.50551,-0.06459,0.42665,-0.19894,0.18624,0.10740,0.27053,0.00000,0.18624,-0.10740,0.01013,0.21962,-0.00000,-0.01898,-0.00005,0.02036,-0.00000,0.00000,-0.00000,0.00000,-0.00130,-0.00015,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,0.00004,0.00000,-0.00000,0.00247,-0.00047,0.00030,0.00000,-0.00000,0.11060,0.00000,-0.00000,-0.00192,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,4080761


In [33]:
fdf['off_def_flag'].value_counts()

none       80342
offense    47383
defense    43389
Name: off_def_flag, dtype: int64

Need to double check the left vs right actions are getting picked up correctly

In [34]:
fdf.agent_type.value_counts()

left     96064
right    75050
Name: agent_type, dtype: int64

## Model Data Prep

To start just get 115 v2 embed

In [53]:
train_off_df = fdf[(fdf.step_num != 0) & (fdf['off_def_flag'] == 'offense')].reset_index(drop=True)
train_off_df = train_off_df[train_off_df.agent_type == 'left'].reset_index(drop=True)
print(train_off_df.shape)
train_off_x_df = train_off_df.iloc[:, 10:125]
train_off_y = train_off_df['action'].values

(28519, 126)


In [39]:
train_off_df['action'].value_counts()

5.00000     13418
4.00000      5162
18.00000     2291
6.00000      2157
0.00000      1439
10.00000      756
13.00000      606
15.00000      570
12.00000      533
3.00000       517
7.00000       402
9.00000       211
8.00000       136
2.00000       125
11.00000       75
1.00000        66
16.00000       30
14.00000       25
Name: action, dtype: int64

In [40]:
train_df['action_str'].value_counts()

right                41849
top_right            24672
release_dribble      18191
bottom_right         16508
idle                 11587
shot                  8787
top_left              7841
left                  6887
top                   6827
bottom_left           6154
bottom                6120
high_pass             4261
release_sprint        3104
sprint                2857
release_direction     1623
sliding               1605
short_pass            1360
long_pass              824
Name: action_str, dtype: int64

## Model Training

In [16]:
#!pip install lightgbm

In [41]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score
def evaluate_model(model,X_test,Y_test):
    preds = model.predict(X_test)
    best_preds = np.asarray([np.argmax(line) for line in preds])

    print("Accuracy = {}".format(accuracy_score(Y_test, best_preds)))
    

In [54]:
%%time
d_train = lgb.Dataset(train_off_x_df, label=train_off_y)
#d_train = lgb.Dataset(np.array(x).reshape((-1,115)), label=y)
params = {}
params['objective'] = 'multiclass'
params['num_classes'] = 19

mod = lgb.train(params, d_train, 100,)

mod.save_model('model.txt')    

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004425 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 23994
[LightGBM] [Info] Number of data points in the train set: 28519, number of used features: 106
[LightGBM] [Info] Start training from score -2.986622
[LightGBM] [Info] Start training from score -6.068671
[LightGBM] [Info] Start training from score -5.430012
[LightGBM] [Info] Start training from score -4.010283
[LightGBM] [Info] Start training from score -1.709246
[LightGBM] [Info] Start training from score -0.753973
[LightGBM] [Info] Start training from score -2.581852
[LightGBM] [Info] Start training from score -4.261874
[LightGBM] [Info] Start training from score -5.345671
[LightGBM] [Info] Start training from score -4.906468
[LightGBM] [Info] Start training from score -3.630284
[LightGBM] [Info] Start training from score -5.9

## Model Evaluation

In [58]:
evaluate_model(mod,train_off_x_df.iloc[-1000:, :],train_off_y[-1000:])

Accuracy = 0.72


In [59]:
evaluate_model(mod, train_off_x_df, train_off_y)

Accuracy = 0.6937830919737719


## Feature Importance

In [60]:
feat_imp_df = pd.DataFrame(list(zip(mod.feature_name(), mod.feature_importance())), 
                          columns=['feature', 'importance'])
feat_imp_df['importance_gains'] = mod.feature_importance(importance_type='gain')
feat_imp_df.sort_values('importance', ascending=False).head(10)

,feature,importance,importance_gains
91,ball_x_dir,1373,274220305.06439
92,ball_y_dir,1232,196431089.69105
57,r_y6,1176,1686014268.50378
90,ball_z_pos,1106,86792229.17280
13,l_y6,1059,405349397.94947
89,ball_y_pos,991,51263599.20677
59,r_y7,979,241793595.33771
93,ball_z_dir,967,10166828.03587
3,l_y1,935,126720495.78232
88,ball_x_pos,875,87475508.24858


## Create Agent to Play

In [61]:
def tree_agent(obs):
    try:
        obs1=obs['observation']['players_raw'][0]
    except:
        obs1=obs['players_raw'][0]

    obs1=convert_observation([obs1])

#     action=np.random.choice(np.arange(19),p=(mod.predict([obs1])).flatten())
    action=np.argmax(mod.predict([obs1]).flatten())
    return [int(action)]

In [66]:
from kaggle_environments import make

env = make("football", configuration={"save_video": True, "episodeSteps": 1000, "scenario_name": "academy_run_to_score_with_keeper", "running_in_notebook": True})
output = env.run([tree_agent,'../submit_agents/tunable-baseline-bot/submission_v6.py'])
print('Left player: reward = %s, status = %s, info = %s' % (output[-1][0]['reward'],
                                                            output[-1][0]['status'], output[-1][0]['info']))
print('Right player: reward = %s, status = %s, info = %s' % (output[-1][1]['reward'],
                                                            output[-1][1]['status'], output[-1][1]['info']))
env.render(mode='human',width=800, height=600)

Left player: reward = 0, status = DONE, info = {}
Right player: reward = 0, status = DONE, info = {}


In [65]:
action_vals = []
for i in range(2800):
    action = output[i][0]['action']
    action_vals.append(action)

IndexError: list index out of range

In [26]:
pd.DataFrame(action_vals).value_counts()

5.00000     1180
4.00000      375
6.00000      255
16.00000     178
18.00000     136
12.00000     115
14.00000     108
2.00000       83
0.00000       75
7.00000       53
9.00000       46
3.00000       46
11.00000      43
1.00000       35
8.00000       34
13.00000      17
15.00000      12
10.00000       8
dtype: int64